In [2]:
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, LayerNormalization, Dropout, Input, MultiHeadAttention
from tensorflow.keras.optimizers import Adam

from captum.attr import GradientShap

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder as le, MinMaxScaler as MM
from pycaret.regression import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from scipy.stats import randint as sp_randint

In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

data = pd.read_excel('/Users/gim-yeon-u/Desktop/Sejong Univ/2024-1/캡스톤/통합 문서4.xlsx')


In [27]:
data

,Month,Day,Hour,Min,Room,Electricity_Usage_kWh
0,3,1,0,0,1,28.268832
1,3,1,0,1,1,16.498430
2,3,1,0,2,1,25.954633
3,3,1,0,3,1,15.041049
4,3,1,0,4,1,10.570874
...,...,...,...,...,...,...
1048569,5,26,4,9,4,71.076569
1048570,5,26,4,10,4,103.324872
1048571,5,26,4,11,4,83.785497
1048572,5,26,4,12,4,17.334466


In [17]:
#data.to_csv('elecdummy.csv')

In [29]:

target = 'Electricity_Usage_kWh'


X = data.drop(target, axis=1).values
y = data[target].values.reshape(-1, 1)


scaler_X = RobustScaler()
scaler_y = RobustScaler()


X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)


X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)


X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.FloatTensor(y_test)


train_data = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(dataset=train_data, batch_size=32, shuffle=True)


class TransformerModel(nn.Module):
    def __init__(self, input_dim, model_dim, num_heads, num_encoder_layers, num_decoder_layers, output_dim):
        super(TransformerModel, self).__init__()
        self.input_linear = nn.Linear(input_dim, model_dim)
        self.transformer = nn.Transformer(
            d_model=model_dim,
            nhead=num_heads,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers
        )
        self.output_linear = nn.Linear(model_dim, output_dim)
    
    def forward(self, src):
        src = src.unsqueeze(1)
        src = self.input_linear(src)
        src = src.permute(1, 0, 2)
        output = self.transformer.encoder(src)
        output = self.output_linear(output.permute(1, 0, 2).squeeze(1))
        return output



model = TransformerModel(input_dim=5, model_dim=128, num_heads=2, num_encoder_layers=2, num_decoder_layers=2, output_dim=1)
optimizer = optim.Adam(model.parameters(), lr=0.0003)
criterion = nn.SmoothL1Loss()


num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()
        predictions = model(batch_x)
        loss = criterion(predictions, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    avg_loss = total_loss / len(train_loader)
    
    model.eval()
    with torch.no_grad():
        val_predictions = model(X_test_tensor)
        val_loss = criterion(val_predictions, y_test_tensor)

    print(f'Epoch {epoch+1}, Training Loss: {avg_loss:.4f}, Validation Loss: {val_loss.item():.4f}')

Epoch 1, Training Loss: 0.2428, Validation Loss: 0.2404
Epoch 2, Training Loss: 0.2417, Validation Loss: 0.2400
Epoch 3, Training Loss: 0.2404, Validation Loss: 0.2369
Epoch 4, Training Loss: 0.2381, Validation Loss: 0.2369
Epoch 5, Training Loss: 0.2365, Validation Loss: 0.2342
Epoch 6, Training Loss: 0.2342, Validation Loss: 0.2316
Epoch 7, Training Loss: 0.2324, Validation Loss: 0.2294
Epoch 8, Training Loss: 0.2309, Validation Loss: 0.2270
Epoch 9, Training Loss: 0.2296, Validation Loss: 0.2255
Epoch 10, Training Loss: 0.2286, Validation Loss: 0.2246
Epoch 11, Training Loss: 0.2273, Validation Loss: 0.2255
Epoch 12, Training Loss: 0.2264, Validation Loss: 0.2225
Epoch 13, Training Loss: 0.2255, Validation Loss: 0.2214
Epoch 14, Training Loss: 0.2247, Validation Loss: 0.2212
Epoch 15, Training Loss: 0.2237, Validation Loss: 0.2224
Epoch 16, Training Loss: 0.2230, Validation Loss: 0.2223
Epoch 17, Training Loss: 0.2222, Validation Loss: 0.2181
Epoch 18, Training Loss: 0.2212, Validat

In [30]:
X_original_tensor = torch.FloatTensor(X_scaled)

In [31]:
model.eval()
with torch.no_grad():
    predictions_tensor = model(X_original_tensor)

In [32]:
predictions = scaler_y.inverse_transform(predictions_tensor.numpy())

In [33]:
predict_excel = pd.DataFrame(predictions)
predict_excel

,0
0,17.466539
1,17.464962
2,17.463379
3,17.461800
4,17.460224
...,...
1048569,61.520790
1048570,61.613380
1048571,61.705940
1048572,61.798519


In [34]:
predict_excel.to_excel('elecprotoresult3.xlsx')

In [35]:
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, 'elecmodelprotorobustfinal2.pth')

In [66]:
import pandas as pd
import torch
from torch import nn
from sklearn.preprocessing import RobustScaler


data_path = '/Users/gim-yeon-u/Desktop/Sejong Univ/2024-1/캡스톤/elecdummyvalidation.xlsx'
data = pd.read_excel(data_path)


X = data.values


scaler_X = RobustScaler()
X_scaled = scaler_X.fit_transform(X)


X_tensor = torch.FloatTensor(X_scaled).unsqueeze(1) 


class TransformerModel(nn.Module):
    def __init__(self, input_dim, model_dim, num_heads, num_encoder_layers, num_decoder_layers, output_dim):
        super(TransformerModel, self).__init__()
        self.input_linear = nn.Linear(input_dim, model_dim)
        self.transformer = nn.Transformer(
            d_model=model_dim,
            nhead=num_heads,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers
        )
        self.output_linear = nn.Linear(model_dim, output_dim)

    def forward(self, src):
        src = self.input_linear(src)
        src = src.squeeze(1)
        src = src.unsqueeze(0)
        output = self.transformer.encoder(src)
        output = self.output_linear(output.squeeze(0))
        return output

model_path = '/Users/gim-yeon-u/Desktop/Sejong Univ/2024-1/캡스톤/elecmodelprotorobustfinal2.pth'
model = TransformerModel(input_dim=5, model_dim=128, num_heads=2, num_encoder_layers=2, num_decoder_layers=2, output_dim=1)
checkpoint = torch.load(model_path)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

with torch.no_grad():
    prediction = model(X_tensor)

predicted_values = prediction.cpu().numpy()

print("Predicted Values:", predicted_values)


Predicted Values: [[-0.19892891]
 [-0.19896285]
 [-0.19899628]
 ...
 [-0.26800784]
 [-0.26801014]
 [-0.26801175]]


In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
import pandas as pd

class TransformerModel(nn.Module):
    def __init__(self, input_dim, model_dim, num_heads, num_encoder_layers, num_decoder_layers, output_dim):
        super(TransformerModel, self).__init__()
        self.input_linear = nn.Linear(input_dim, model_dim)
        self.transformer = nn.Transformer(
            d_model=model_dim,
            nhead=num_heads,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers
        )
        self.output_linear = nn.Linear(model_dim, output_dim)
    
    def forward(self, src):
        src = src.unsqueeze(1)
        src = self.input_linear(src)
        src = src.permute(1, 0, 2)
        output = self.transformer.encoder(src)
        output = self.output_linear(output.permute(1, 0, 2).squeeze(1))
        return output

model_path = '/Users/gim-yeon-u/Desktop/Sejong Univ/2024-1/캡스톤/elecmodelprotorobustfinal2.pth'
model = TransformerModel(input_dim=5, model_dim=128, num_heads=2, num_encoder_layers=2, num_decoder_layers=2, output_dim=1)
checkpoint = torch.load(model_path)
model.load_state_dict(checkpoint['model_state_dict'])

data_path = '/Users/gim-yeon-u/Desktop/Sejong Univ/2024-1/캡스톤/elecdummy3.xlsx'
data = pd.read_excel(data_path) 
target = 'Electricity_Usage_kWh'
X = data.drop(target, axis=1).values
y = data[target].values.reshape(-1, 1)

scaler_X = RobustScaler()
scaler_y = RobustScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.FloatTensor(y_test)

train_data = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(dataset=train_data, batch_size=32, shuffle=True)

optimizer = optim.Adam(model.parameters(), lr=0.0003)
criterion = nn.SmoothL1Loss()

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()
        predictions = model(batch_x)
        loss = criterion(predictions, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    avg_loss = total_loss / len(train_loader)
    
    model.eval()
    with torch.no_grad():
        val_predictions = model(X_test_tensor)
        val_loss = criterion(val_predictions, y_test_tensor)

    print(f'Epoch {epoch+1}, Training Loss: {avg_loss:.4f}, Validation Loss: {val_loss.item():.4f}')

torch.save({'model_state_dict': model.state_dict()}, 'fine_tuned_model_elec_final.pth')

/Users/gim-yeon-u/miniforge3/envs/tf25/lib/python3.8/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1, Training Loss: 0.1807, Validation Loss: 0.1713
Epoch 2, Training Loss: 0.1728, Validation Loss: 0.1714
Epoch 3, Training Loss: 0.1707, Validation Loss: 0.1656
Epoch 4, Training Loss: 0.1685, Validation Loss: 0.1632
Epoch 5, Training Loss: 0.1665, Validation Loss: 0.1600
Epoch 6, Training Loss: 0.1648, Validation Loss: 0.1600
Epoch 7, Training Loss: 0.1639, Validation Loss: 0.1606
Epoch 8, Training Loss: 0.1630, Validation Loss: 0.1574
Epoch 9, Training Loss: 0.1622, Validation Loss: 0.1571
Epoch 10, Training Loss: 0.1616, Validation Loss: 0.1586


In [12]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from joblib import dump, load

class TransformerModel(nn.Module):
    def __init__(self, input_dim, model_dim, num_heads, num_encoder_layers, num_decoder_layers, output_dim):
        super(TransformerModel, self).__init__()
        self.input_linear = nn.Linear(input_dim, model_dim)
        self.transformer = nn.Transformer(
            d_model=model_dim,
            nhead=num_heads,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers
        )
        self.output_linear = nn.Linear(model_dim, output_dim)
    
    def forward(self, src):
        src = self.input_linear(src)
        src = src.permute(1, 0, 2)  
        output = self.transformer.encoder(src)
        output = self.output_linear(output.permute(1, 0, 2).squeeze(1))
        return output

train_data_path = '/Users/gim-yeon-u/Desktop/Sejong Univ/2024-1/캡스톤/elecdummy3.xlsx'
train_data = pd.read_excel(train_data_path)
X_train = train_data.drop('Electricity_Usage_kWh', axis=1).values
y_train = train_data['Electricity_Usage_kWh'].values.reshape(-1, 1)

scaler_X = RobustScaler()
scaler_y = RobustScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
y_train_scaled = scaler_y.fit_transform(y_train)

dump(scaler_X, 'scaler_X.joblib')
dump(scaler_y, 'scaler_y.joblib')


model_path = '/Users/gim-yeon-u/Desktop/Sejong Univ/2024-1/캡스톤/fine_tuned_model_elec_final.pth'
model = TransformerModel(input_dim=5, model_dim=128, num_heads=2, num_encoder_layers=2, num_decoder_layers=2, output_dim=1)
checkpoint = torch.load(model_path)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

predict_data_path = '/Users/gim-yeon-u/Desktop/Sejong Univ/2024-1/캡스톤/elec5.xlsx'
predict_data = pd.read_excel(predict_data_path)
X_predict = predict_data.values

scaler_X = load('scaler_X.joblib')
X_predict_scaled = scaler_X.transform(X_predict)
X_predict_tensor = torch.FloatTensor(X_predict_scaled).unsqueeze(1)

with torch.no_grad():
    prediction_tensor = model(X_predict_tensor)
    prediction = prediction_tensor.cpu().numpy()

scaler_y = load('scaler_y.joblib')
predicted_values_rescaled = scaler_y.inverse_transform(prediction)

print("Predicted Values:", predicted_values_rescaled)

/Users/gim-yeon-u/miniforge3/envs/tf25/lib/python3.8/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Predicted Values: [[17.391598 ]
 [17.390575 ]
 [17.389557 ]
 ...
 [ 6.5350723]
 [ 6.535635 ]
 [ 6.5362053]]


In [16]:
model_path = '/Users/gim-yeon-u/Desktop/Sejong Univ/2024-1/캡스톤/fine_tuned_model_elec_final.pth'
model = TransformerModel(input_dim=5, model_dim=128, num_heads=2, num_encoder_layers=2, num_decoder_layers=2, output_dim=1)
checkpoint = torch.load(model_path)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

predict_data_path = '/Users/gim-yeon-u/Desktop/Sejong Univ/2024-1/캡스톤/elec5.xlsx'
predict_data = pd.read_excel(predict_data_path)
X_predict = predict_data.values

scaler_X = load('scaler_X.joblib')
X_predict_scaled = scaler_X.transform(X_predict)
X_predict_tensor = torch.FloatTensor(X_predict_scaled).unsqueeze(1)

with torch.no_grad():
    prediction_tensor = model(X_predict_tensor)
    prediction = prediction_tensor.cpu().numpy()

scaler_y = load('scaler_y.joblib')
predicted_values_rescaled = scaler_y.inverse_transform(prediction)

print("Predicted Values:", predicted_values_rescaled)

/Users/gim-yeon-u/miniforge3/envs/tf25/lib/python3.8/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Predicted Values: [[56.982906]
 [57.003807]
 [57.024197]
 ...
 [10.696982]
 [10.702027]
 [10.707076]]


In [17]:
data_final = pd.DataFrame(predicted_values_rescaled)

In [19]:
data_final.to_csv('elecresultfinal2.xlsx')